In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

## Data Preparation

In [1]:
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import RandomizedSearchCV, train_test_split;
import numpy;
import pandas;
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier;
import sys;
from keras.utils import to_categorical;
# REMOTE_MODULE_PATH = '/home/macbookretina/automatic-music-genre-classification/feature_extraction_deep_learning'
LOCAL_MODULE_PATH = '/Users/macbookretina/Desktop/automatic-music-genre-classification/feature_extraction_deep_learning'
sys.path.insert(1, LOCAL_MODULE_PATH)
from custom_module.utilities import *

Using TensorFlow backend.
/Users/macbookretina/Library/Python/3.7/lib/python/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/macbookretina/Library/Python/3.7/lib/python/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
features = pandas.read_csv(LOCAL_MOUNTED_DATASET_PATH + '/data/cleaned_features.csv', index_col=0)
features

,genre_label,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,3,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,3,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [3]:
y = features.loc[:,'genre_label']
X = features.loc[:, features.columns.difference(['genre_label'])]
X

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,mean_db4_cD3,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,-0.000069,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,0.000984,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,0.000019,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,-0.001993,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,0.000076,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,-0.000065,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,-0.000009,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,-0.000024,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,-0.000132,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [4]:
y.value_counts(normalize=True)

3    0.333434
1    0.333434
2    0.333131
Name: genre_label, dtype: float64

In [5]:
wavelet_predictors = X.filter(regex=(r'.+_db[458]{1}_.+'))
wavelet_predictor_labels = wavelet_predictors.columns.values

In [6]:
preprocess_sp = standardization_pipeline(
    wavelet_predictor_labels,
    [], wavelet_predictor_labels)
preprocess_np = normalization_pipeline(
    wavelet_predictor_labels)

### Split & Transform Data

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    wavelet_predictors, y, test_size=0.2, shuffle=True)

In [8]:
# X_train_sp = standardization_pipeline(wavelet_predictor_labels, 
#                                    [], wavelet_predictor_labels
#                                   ).fit_transform(X_train)
X_test_sp = standardization_pipeline(wavelet_predictor_labels, 
                                   [], wavelet_predictor_labels
                                  ).fit_transform(X_test)
# X_train_sp.shape, X_test_sp.shape

In [9]:
# X_train_np = normalization_pipeline(wavelet_predictor_labels
#                                   ).fit_transform(X_train)
X_test_np = normalization_pipeline(wavelet_predictor_labels
                                  ).fit_transform(X_test)
# X_train_np.shape, X_test_np.shape

In [10]:
# nrows, ncols = X_train_sp.shape
# X_train_sp = X_train_sp.reshape(nrows, ncols, 1)
# X_train_np = X_train_np.reshape(nrows, ncols, 1)

In [11]:
# nrows, ncols = X_test_sp.shape
# X_test_sp = X_test_sp.reshape(nrows, ncols, 1)
# X_test_np = X_test_np.reshape(nrows, ncols, 1)

In [12]:
_, ncols = X_train.shape

In [13]:
# one hot encoding our labels
y_train = to_categorical(y_train, 4)
y_test = to_categorical(y_test, 4)

In [14]:
y_train.shape, y_test.shape

((2639, 4), (660, 4))

## Predictive Modelling

In [15]:
early_stopping_cb = callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)

In [16]:
param_dist = {
    'clf__n_hiddden': [0,1,2,3,4,5],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100)
}

### FNN With Standardized Input

In [19]:
# wrap the function with keras wrapper
fnn_1 = KerasClassifier(build_fn=set_shape_create_model('fnn', ncols))

In [20]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', fnn_1)
])

In [21]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [22]:
# fit model
search_1 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 108)               0         
_________________________________________________________________
dense (Dense)                (None, 74)                8066      
_________________________________________________________________
dense_1 (Dense)              (None, 74)                5550      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 300       
Total params: 13,916
Trainable params: 13,916
Non-trainable params: 0
_________________________________________________________________
None
Train on 1759 samples, validate on 660 samples
Epoch 1/100
1759/1759 [==============================] - 1s 729us/sample - loss: 1.1603 - accuracy: 0.4474 - val_loss: 1.0044 - val_accuracy: 0.5182
Epoch 2/100
1759/1759 [==============================] -

Epoch 3/100
1760/1760 [==============================] - 0s 147us/sample - loss: 0.8120 - accuracy: 0.6381 - val_loss: 0.9365 - val_accuracy: 0.5545
Epoch 4/100
1760/1760 [==============================] - 0s 153us/sample - loss: 0.7536 - accuracy: 0.6790 - val_loss: 0.9444 - val_accuracy: 0.5591
Epoch 5/100
1760/1760 [==============================] - 0s 156us/sample - loss: 0.7166 - accuracy: 0.6852 - val_loss: 0.9144 - val_accuracy: 0.5727
Epoch 6/100
1760/1760 [==============================] - 0s 175us/sample - loss: 0.6772 - accuracy: 0.7153 - val_loss: 0.9202 - val_accuracy: 0.5818
Epoch 7/100
1760/1760 [==============================] - 0s 137us/sample - loss: 0.6368 - accuracy: 0.7318 - val_loss: 0.9303 - val_accuracy: 0.5818
Epoch 8/100
1760/1760 [==============================] - 0s 135us/sample - loss: 0.6086 - accuracy: 0.7511 - val_loss: 0.9242 - val_accuracy: 0.5879
Epoch 9/100
1760/1760 [==============================] - 0s 139us/sample - loss: 0.5779 - accuracy: 0.7557

1759/1759 [==============================] - 0s 129us/sample - loss: 0.8874 - accuracy: 0.6180 - val_loss: 0.9846 - val_accuracy: 0.5485
Epoch 16/100
1759/1759 [==============================] - 0s 117us/sample - loss: 0.8807 - accuracy: 0.6174 - val_loss: 0.9813 - val_accuracy: 0.5470
Epoch 17/100
1759/1759 [==============================] - 0s 120us/sample - loss: 0.8746 - accuracy: 0.6185 - val_loss: 0.9783 - val_accuracy: 0.5485
Epoch 18/100
1759/1759 [==============================] - 0s 135us/sample - loss: 0.8689 - accuracy: 0.6248 - val_loss: 0.9758 - val_accuracy: 0.5485
Epoch 19/100
1759/1759 [==============================] - 0s 143us/sample - loss: 0.8636 - accuracy: 0.6254 - val_loss: 0.9732 - val_accuracy: 0.5515
Epoch 20/100
1759/1759 [==============================] - 0s 105us/sample - loss: 0.8587 - accuracy: 0.6305 - val_loss: 0.9714 - val_accuracy: 0.5561
Epoch 21/100
1759/1759 [==============================] - 0s 107us/sample - loss: 0.8540 - accuracy: 0.6316 - val

Epoch 70/100
1759/1759 [==============================] - 0s 149us/sample - loss: 0.7479 - accuracy: 0.6816 - val_loss: 0.9314 - val_accuracy: 0.5833
Epoch 71/100
1759/1759 [==============================] - 0s 98us/sample - loss: 0.7468 - accuracy: 0.6816 - val_loss: 0.9306 - val_accuracy: 0.5818
Epoch 72/100
1759/1759 [==============================] - 0s 99us/sample - loss: 0.7454 - accuracy: 0.6822 - val_loss: 0.9302 - val_accuracy: 0.5848
Epoch 73/100
1759/1759 [==============================] - 0s 103us/sample - loss: 0.7442 - accuracy: 0.6828 - val_loss: 0.9299 - val_accuracy: 0.5864
Epoch 74/100
1759/1759 [==============================] - 0s 101us/sample - loss: 0.7430 - accuracy: 0.6799 - val_loss: 0.9296 - val_accuracy: 0.5879
Epoch 75/100
1759/1759 [==============================] - 0s 97us/sample - loss: 0.7418 - accuracy: 0.6816 - val_loss: 0.9295 - val_accuracy: 0.5879
Epoch 76/100
1759/1759 [==============================] - 0s 133us/sample - loss: 0.7407 - accuracy: 0.

Epoch 18/100
1759/1759 [==============================] - 0s 74us/sample - loss: 0.8696 - accuracy: 0.6276 - val_loss: 0.9678 - val_accuracy: 0.5545
Epoch 19/100
1759/1759 [==============================] - 0s 75us/sample - loss: 0.8638 - accuracy: 0.6345 - val_loss: 0.9650 - val_accuracy: 0.5545
Epoch 20/100
1759/1759 [==============================] - 0s 71us/sample - loss: 0.8586 - accuracy: 0.6373 - val_loss: 0.9626 - val_accuracy: 0.5591
Epoch 21/100
1759/1759 [==============================] - 0s 70us/sample - loss: 0.8536 - accuracy: 0.6379 - val_loss: 0.9607 - val_accuracy: 0.5576
Epoch 22/100
1759/1759 [==============================] - 0s 96us/sample - loss: 0.8490 - accuracy: 0.6407 - val_loss: 0.9583 - val_accuracy: 0.5591
Epoch 23/100
1759/1759 [==============================] - 0s 106us/sample - loss: 0.8445 - accuracy: 0.6418 - val_loss: 0.9562 - val_accuracy: 0.5576
Epoch 24/100
1759/1759 [==============================] - 0s 117us/sample - loss: 0.8405 - accuracy: 0.64

...

Epoch 72/100
1759/1759 [==============================] - 0s 124us/sample - loss: 1.2352 - accuracy: 0.4719 - val_loss: 1.3075 - val_accuracy: 0.4636
Epoch 73/100
1759/1759 [==============================] - 0s 132us/sample - loss: 1.2334 - accuracy: 0.4719 - val_loss: 1.3062 - val_accuracy: 0.4636
Epoch 74/100
1759/1759 [==============================] - 0s 79us/sample - loss: 1.2317 - accuracy: 0.4719 - val_loss: 1.3049 - val_accuracy: 0.4636
Epoch 75/100
1759/1759 [==============================] - 0s 80us/sample - loss: 1.2299 - accuracy: 0.4736 - val_loss: 1.3036 - val_accuracy: 0.4636
Epoch 76/100
1759/1759 [==============================] - 0s 107us/sample - loss: 1.2282 - accuracy: 0.4758 - val_loss: 1.3024 - val_accuracy: 0.4667
Epoch 77/100
1759/1759 [==============================] - 0s 109us/sample - loss: 1.2265 - accuracy: 0.4775 - val_loss: 1.3011 - val_accuracy: 0.4682
Epoch 78/100
1759/1759 [==============================] - 0s 95us/sample - loss: 1.2248 - accuracy: 0.

Epoch 22/100
1759/1759 [==============================] - 0s 85us/sample - loss: 1.4115 - accuracy: 0.3684 - val_loss: 1.4229 - val_accuracy: 0.3758
Epoch 23/100
1759/1759 [==============================] - 0s 86us/sample - loss: 1.4054 - accuracy: 0.3724 - val_loss: 1.4180 - val_accuracy: 0.3758
Epoch 24/100
1759/1759 [==============================] - 0s 81us/sample - loss: 1.3995 - accuracy: 0.3764 - val_loss: 1.4133 - val_accuracy: 0.3803
Epoch 25/100
1759/1759 [==============================] - 0s 83us/sample - loss: 1.3939 - accuracy: 0.3786 - val_loss: 1.4088 - val_accuracy: 0.3909
Epoch 26/100
1759/1759 [==============================] - 0s 79us/sample - loss: 1.3885 - accuracy: 0.3803 - val_loss: 1.4045 - val_accuracy: 0.3924
Epoch 27/100
1759/1759 [==============================] - 0s 83us/sample - loss: 1.3833 - accuracy: 0.3843 - val_loss: 1.4003 - val_accuracy: 0.3970
Epoch 28/100
1759/1759 [==============================] - 0s 87us/sample - loss: 1.3783 - accuracy: 0.3860

Epoch 77/100
1759/1759 [==============================] - 0s 60us/sample - loss: 1.2373 - accuracy: 0.4775 - val_loss: 1.2872 - val_accuracy: 0.4742
Epoch 78/100
1759/1759 [==============================] - 0s 57us/sample - loss: 1.2355 - accuracy: 0.4804 - val_loss: 1.2859 - val_accuracy: 0.4742
Epoch 79/100
1759/1759 [==============================] - 0s 65us/sample - loss: 1.2338 - accuracy: 0.4815 - val_loss: 1.2846 - val_accuracy: 0.4758
Epoch 80/100
1759/1759 [==============================] - 0s 64us/sample - loss: 1.2321 - accuracy: 0.4832 - val_loss: 1.2833 - val_accuracy: 0.4758
Epoch 81/100
1759/1759 [==============================] - 0s 58us/sample - loss: 1.2304 - accuracy: 0.4844 - val_loss: 1.2820 - val_accuracy: 0.4742
Epoch 82/100
1759/1759 [==============================] - 0s 61us/sample - loss: 1.2287 - accuracy: 0.4855 - val_loss: 1.2807 - val_accuracy: 0.4788
Epoch 83/100
1759/1759 [==============================] - 0s 58us/sample - loss: 1.2270 - accuracy: 0.4861

Epoch 27/100
1760/1760 [==============================] - 0s 61us/sample - loss: 1.3840 - accuracy: 0.3858 - val_loss: 1.4092 - val_accuracy: 0.4076
Epoch 28/100
1760/1760 [==============================] - 0s 59us/sample - loss: 1.3790 - accuracy: 0.3875 - val_loss: 1.4054 - val_accuracy: 0.4106
Epoch 29/100
1760/1760 [==============================] - 0s 64us/sample - loss: 1.3741 - accuracy: 0.3886 - val_loss: 1.4018 - val_accuracy: 0.4121
Epoch 30/100
1760/1760 [==============================] - 0s 57us/sample - loss: 1.3694 - accuracy: 0.3926 - val_loss: 1.3983 - val_accuracy: 0.4121
Epoch 31/100
1760/1760 [==============================] - 0s 55us/sample - loss: 1.3649 - accuracy: 0.3960 - val_loss: 1.3949 - val_accuracy: 0.4121
Epoch 32/100
1760/1760 [==============================] - 0s 57us/sample - loss: 1.3605 - accuracy: 0.3983 - val_loss: 1.3916 - val_accuracy: 0.4212
Epoch 33/100
1760/1760 [==============================] - 0s 76us/sample - loss: 1.3562 - accuracy: 0.3989

Epoch 82/100
1760/1760 [==============================] - 0s 96us/sample - loss: 1.2308 - accuracy: 0.4920 - val_loss: 1.2979 - val_accuracy: 0.4682
Epoch 83/100
1760/1760 [==============================] - 0s 99us/sample - loss: 1.2292 - accuracy: 0.4932 - val_loss: 1.2968 - val_accuracy: 0.4682
Epoch 84/100
1760/1760 [==============================] - 0s 92us/sample - loss: 1.2276 - accuracy: 0.4949 - val_loss: 1.2957 - val_accuracy: 0.4667
Epoch 85/100
1760/1760 [==============================] - 0s 94us/sample - loss: 1.2260 - accuracy: 0.4972 - val_loss: 1.2946 - val_accuracy: 0.4667
Epoch 86/100
1760/1760 [==============================] - 0s 102us/sample - loss: 1.2244 - accuracy: 0.4983 - val_loss: 1.2935 - val_accuracy: 0.4682
Epoch 87/100
1760/1760 [==============================] - 0s 97us/sample - loss: 1.2229 - accuracy: 0.4983 - val_loss: 1.2924 - val_accuracy: 0.4682
Epoch 88/100
1760/1760 [==============================] - 0s 100us/sample - loss: 1.2214 - accuracy: 0.49

In [26]:
search_1.best_score_

0.6710875447431538

In [27]:
search_1.best_params_

{'clf__optimizer': 'rmsprop',
 'clf__n_neurons': 79,
 'clf__n_hiddden': 1,
 'clf__kernel_initializer': 'uniform',
 'clf__activation': 'selu'}

In [29]:
search_1.best_estimator_

Pipeline(memory=None,
         steps=[('preprocess',
                 Pipeline(memory=None,
                          steps=[('addcolumnnames',
                                  AddColumnNames(columns=array(['mean_db4_cA4', 'mean_db4_cD1', 'mean_db4_cD2', 'mean_db4_cD3',
       'mean_db4_cD4', 'mean_db5_cA4', 'mean_db5_cD1', 'mean_db5_cD2',
       'mean_db5_cD3', 'mean_db5_cD4', 'mean_db8_cA7', 'mean_db8_cD1',
       'mean_db8_cD2', 'mean_db8_cD3', 'mean_db8_cD4', 'mean_d...
       'var_db8_cD3', 'var_db8_cD4', 'var_db8_cD5', 'var_db8_cD6',
       'var_db8_cD7'], dtype=object))),
                                                                                  ('standardscaler',
                                                                                   StandardScaler(copy=True,
                                                                                                  with_mean=True,
                                                                                         

### FNN With Normalized Input

In [30]:
# wrap the function with keras wrapper
fnn_2 = KerasClassifier(set_shape_create_model('fnn_np', ncols))

In [31]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', fnn_2)
])

In [32]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [33]:
# fit model
search_2 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_32 (Flatten)         (None, 108)               0         
_________________________________________________________________
dense_100 (Dense)            (None, 4)                 436       
Total params: 436
Trainable params: 436
Non-trainable params: 0
_________________________________________________________________
None
Train on 1759 samples, validate on 660 samples
Epoch 1/100
1759/1759 [==============================] - 1s 374us/sample - loss: 1.3695 - accuracy: 0.2564 - val_loss: 1.1868 - val_accuracy: 0.3727
Epoch 2/100
1759/1759 [==============================] - 0s 91us/sample - loss: 1.1517 - accuracy: 0.3815 - val_loss: 1.1064 - val_accuracy: 0.4167
Epoch 3/100
1759/1759 [==============================] - 0s 100us/sample - loss: 1.1032 - accuracy: 0.4480 - val_loss: 1.0805 - val_accuracy: 0.4136
Epoch 4/100
1759/1759 [===

1759/1759 [==============================] - 0s 56us/sample - loss: 1.0556 - accuracy: 0.5043 - val_loss: 1.0858 - val_accuracy: 0.3606
Epoch 22/100
1759/1759 [==============================] - 0s 56us/sample - loss: 1.0539 - accuracy: 0.5105 - val_loss: 1.0836 - val_accuracy: 0.3712
Epoch 23/100
1759/1759 [==============================] - 0s 56us/sample - loss: 1.0523 - accuracy: 0.5060 - val_loss: 1.0815 - val_accuracy: 0.3742
Epoch 24/100
1759/1759 [==============================] - 0s 58us/sample - loss: 1.0504 - accuracy: 0.5077 - val_loss: 1.0830 - val_accuracy: 0.3682
Epoch 25/100
1759/1759 [==============================] - 0s 61us/sample - loss: 1.0488 - accuracy: 0.5117 - val_loss: 1.0798 - val_accuracy: 0.3758
Epoch 26/100
1759/1759 [==============================] - 0s 58us/sample - loss: 1.0472 - accuracy: 0.5128 - val_loss: 1.0798 - val_accuracy: 0.3727
Epoch 27/100
1759/1759 [==============================] - 0s 58us/sample - loss: 1.0456 - accuracy: 0.5111 - val_loss: 

1760/1760 [==============================] - 0s 53us/sample - loss: 1.0473 - accuracy: 0.5222 - val_loss: 1.0822 - val_accuracy: 0.4061
Epoch 33/100
1760/1760 [==============================] - 0s 56us/sample - loss: 1.0457 - accuracy: 0.5233 - val_loss: 1.0817 - val_accuracy: 0.4030
Epoch 34/100
1760/1760 [==============================] - 0s 59us/sample - loss: 1.0445 - accuracy: 0.5233 - val_loss: 1.0805 - val_accuracy: 0.4030
Epoch 35/100
1760/1760 [==============================] - 0s 55us/sample - loss: 1.0431 - accuracy: 0.5210 - val_loss: 1.0784 - val_accuracy: 0.4091
Epoch 36/100
1760/1760 [==============================] - 0s 54us/sample - loss: 1.0416 - accuracy: 0.5267 - val_loss: 1.0780 - val_accuracy: 0.4106
Epoch 37/100
1760/1760 [==============================] - 0s 54us/sample - loss: 1.0403 - accuracy: 0.5295 - val_loss: 1.0748 - val_accuracy: 0.4136
Epoch 38/100
1760/1760 [==============================] - 0s 59us/sample - loss: 1.0389 - accuracy: 0.5273 - val_loss: 

1760/1760 [==============================] - 0s 73us/sample - loss: 0.9871 - accuracy: 0.5591 - val_loss: 1.0521 - val_accuracy: 0.4273
Epoch 88/100
1760/1760 [==============================] - 0s 87us/sample - loss: 0.9864 - accuracy: 0.5597 - val_loss: 1.0510 - val_accuracy: 0.4333
Epoch 89/100
1760/1760 [==============================] - 0s 80us/sample - loss: 0.9858 - accuracy: 0.5597 - val_loss: 1.0500 - val_accuracy: 0.4333
Epoch 90/100
1760/1760 [==============================] - 0s 74us/sample - loss: 0.9852 - accuracy: 0.5608 - val_loss: 1.0492 - val_accuracy: 0.4394
Epoch 91/100
1760/1760 [==============================] - 0s 72us/sample - loss: 0.9845 - accuracy: 0.5591 - val_loss: 1.0491 - val_accuracy: 0.4364
Epoch 92/100
1760/1760 [==============================] - 0s 71us/sample - loss: 0.9839 - accuracy: 0.5614 - val_loss: 1.0486 - val_accuracy: 0.4394
Epoch 93/100
1760/1760 [==============================] - 0s 57us/sample - loss: 0.9833 - accuracy: 0.5614 - val_loss: 

Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_41 (Flatten)         (None, 108)               0         
_________________________________________________________________
dense_130 (Dense)            (None, 36)                3924      
_________________________________________________________________
dense_131 (Dense)            (None, 4)                 148       
Total params: 4,072
Trainable params: 4,072
Non-trainable params: 0
_________________________________________________________________
None
Train on 1759 samples, validate on 660 samples
Epoch 1/100
1759/1759 [==============================] - 0s 284us/sample - loss: 1.1048 - accuracy: 0.4110 - val_loss: 1.0642 - val_accuracy: 0.4515
Epoch 2/100
1759/1759 [==============================] - 0s 57us/sample - loss: 1.0201 - accuracy: 0.4969 - val_loss: 1.0664 - val_accuracy: 0.4167
Epoch 3/100
1759/1759 [================

...

Epoch 1/100
1760/1760 [==============================] - 1s 328us/sample - loss: 1.1542 - accuracy: 0.3125 - val_loss: 1.1111 - val_accuracy: 0.3424
Epoch 2/100
1760/1760 [==============================] - 0s 68us/sample - loss: 1.1054 - accuracy: 0.3386 - val_loss: 1.1067 - val_accuracy: 0.3424
Epoch 3/100
1760/1760 [==============================] - 0s 68us/sample - loss: 1.1087 - accuracy: 0.3381 - val_loss: 1.1016 - val_accuracy: 0.3197
Epoch 4/100
1760/1760 [==============================] - 0s 69us/sample - loss: 1.1026 - accuracy: 0.3290 - val_loss: 1.0820 - val_accuracy: 0.4455
Epoch 5/100
1760/1760 [==============================] - 0s 68us/sample - loss: 1.0033 - accuracy: 0.4875 - val_loss: 1.1391 - val_accuracy: 0.4227
Epoch 6/100
1760/1760 [==============================] - 0s 67us/sample - loss: 0.9412 - accuracy: 0.5443 - val_loss: 1.0716 - val_accuracy: 0.4500
Epoch 7/100
1760/1760 [==============================] - 0s 67us/sample - loss: 0.9272 - accuracy: 0.5494 - val

Epoch 9/100
1759/1759 [==============================] - 0s 85us/sample - loss: 0.8471 - accuracy: 0.6043 - val_loss: 1.0132 - val_accuracy: 0.5348
Epoch 10/100
1759/1759 [==============================] - 0s 85us/sample - loss: 0.8442 - accuracy: 0.6128 - val_loss: 1.1526 - val_accuracy: 0.4939
Epoch 11/100
1759/1759 [==============================] - 0s 84us/sample - loss: 0.8343 - accuracy: 0.6197 - val_loss: 1.1304 - val_accuracy: 0.5061
Epoch 12/100
880/880 [==============================] - 0s 249us/sample - loss: 0.9385 - accuracy: 0.5705
Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_55 (Flatten)         (None, 108)               0         
_________________________________________________________________
dense_186 (Dense)            (None, 77)                8393      
_________________________________________________________________
dense_187 (Dense)            (None, 

1759/1759 [==============================] - 0s 79us/sample - loss: 1.0985 - accuracy: 0.3388 - val_loss: 1.0978 - val_accuracy: 0.3424
Epoch 78/100
1759/1759 [==============================] - 0s 76us/sample - loss: 1.0984 - accuracy: 0.3371 - val_loss: 1.0978 - val_accuracy: 0.3424
Epoch 79/100
1759/1759 [==============================] - 0s 82us/sample - loss: 1.0985 - accuracy: 0.3434 - val_loss: 1.0979 - val_accuracy: 0.3439
Epoch 80/100
1759/1759 [==============================] - 0s 78us/sample - loss: 1.0984 - accuracy: 0.3735 - val_loss: 1.0979 - val_accuracy: 0.3424
Epoch 81/100
1759/1759 [==============================] - 0s 77us/sample - loss: 1.0984 - accuracy: 0.3422 - val_loss: 1.0979 - val_accuracy: 0.3424
Epoch 82/100
1759/1759 [==============================] - 0s 73us/sample - loss: 1.0984 - accuracy: 0.3360 - val_loss: 1.0978 - val_accuracy: 0.3424
Epoch 83/100
1759/1759 [==============================] - 0s 60us/sample - loss: 1.0984 - accuracy: 0.3366 - val_loss: 

Epoch 78/100
1759/1759 [==============================] - 0s 61us/sample - loss: 1.0967 - accuracy: 0.3377 - val_loss: 1.0973 - val_accuracy: 0.3424
Epoch 79/100
1759/1759 [==============================] - 0s 62us/sample - loss: 1.0966 - accuracy: 0.3388 - val_loss: 1.0973 - val_accuracy: 0.3424
Epoch 80/100
1759/1759 [==============================] - 0s 60us/sample - loss: 1.0966 - accuracy: 0.3644 - val_loss: 1.0972 - val_accuracy: 0.3424
Epoch 81/100
1759/1759 [==============================] - 0s 60us/sample - loss: 1.0964 - accuracy: 0.3388 - val_loss: 1.0972 - val_accuracy: 0.3424
Epoch 82/100
1759/1759 [==============================] - 0s 59us/sample - loss: 1.0964 - accuracy: 0.3434 - val_loss: 1.0971 - val_accuracy: 0.3424
Epoch 83/100
1759/1759 [==============================] - 0s 62us/sample - loss: 1.0964 - accuracy: 0.3565 - val_loss: 1.0970 - val_accuracy: 0.3424
Epoch 84/100
1759/1759 [==============================] - 0s 61us/sample - loss: 1.0963 - accuracy: 0.3451

Epoch 15/100
1759/1759 [==============================] - 0s 72us/sample - loss: 0.8319 - accuracy: 0.5992 - val_loss: 1.1645 - val_accuracy: 0.4788
Epoch 16/100
1759/1759 [==============================] - 0s 71us/sample - loss: 0.8249 - accuracy: 0.6197 - val_loss: 1.2673 - val_accuracy: 0.4621
Epoch 17/100
1759/1759 [==============================] - 0s 70us/sample - loss: 0.8217 - accuracy: 0.6208 - val_loss: 1.1764 - val_accuracy: 0.4682
Epoch 18/100
880/880 [==============================] - 0s 207us/sample - loss: 0.9388 - accuracy: 0.5432
Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_61 (Flatten)         (None, 108)               0         
_________________________________________________________________
dense_215 (Dense)            (None, 67)                7303      
_________________________________________________________________
dense_216 (Dense)            (None,

In [36]:
search_2.best_score_

0.5665024827718284

In [37]:
search_2.best_params_

{'clf__optimizer': 'adam',
 'clf__n_neurons': 76,
 'clf__n_hiddden': 2,
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__activation': 'selu'}

In [38]:
search_2.best_estimator_

Pipeline(memory=None,
         steps=[('preprocess',
                 Pipeline(memory=None,
                          steps=[('addcolumnnames',
                                  AddColumnNames(columns=array(['mean_db4_cA4', 'mean_db4_cD1', 'mean_db4_cD2', 'mean_db4_cD3',
       'mean_db4_cD4', 'mean_db5_cA4', 'mean_db5_cD1', 'mean_db5_cD2',
       'mean_db5_cD3', 'mean_db5_cD4', 'mean_db8_cA7', 'mean_db8_cD1',
       'mean_db8_cD2', 'mean_db8_cD3', 'mean_db8_cD4', 'mean_d...
       'var_db8_cD3', 'var_db8_cD4', 'var_db8_cD5', 'var_db8_cD6',
       'var_db8_cD7'], dtype=object))),
                                                                                  ('minmaxscaler',
                                                                                   MinMaxScaler(copy=True,
                                                                                                feature_range=(0,
                                                                                             

### CNN With Standardized Input

In [22]:
param_dist = {
    'clf__n_hidden': [0,1,2,3,4,5],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100),
    'clf__filters': [16, 32, 64],
    'clf__kernel_size': [3,4,5],
    'clf__dropout':[.25, .5]
}

In [23]:
# wrap the function with keras wrapper
cnn_1 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn', ncols))

In [24]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', cnn_1)
])

In [25]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=4)

In [ ]:
# fit model
search_3 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

In [ ]:
search_3.best_score_

In [ ]:
search_3.best_params_

In [ ]:
search_3.best_estimator_

### CNN With Normalized Input

In [18]:
# wrap the function with keras wrapper
cnn_2 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn_np', ncols))

In [19]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', cnn_1)
])

In [20]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=4)

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__preprocess', 'estimator__clf', 'estimator__preprocess__memory', 'estimator__preprocess__steps', 'estimator__preprocess__verbose', 'estimator__preprocess__addcolumnnames', 'estimator__preprocess__featureunion', 'estimator__preprocess__reshape2d', 'estimator__preprocess__addcolumnnames__columns', 'estimator__preprocess__featureunion__n_jobs', 'estimator__preprocess__featureunion__transformer_list', 'estimator__preprocess__featureunion__transformer_weights', 'estimator__preprocess__featureunion__verbose', 'estimator__preprocess__featureunion__predictors_without_outliers', 'estimator__preprocess__featureunion__predictors_without_outliers__memory', 'estimator__preprocess__featureunion__predictors_without_outliers__steps', 'estimator__preprocess__featureunion__predictors_without_outliers__verbose', 'estimator__preprocess__featureunion__predictors_without_outliers__columnselector', 'esti

In [ ]:
# fit model
search_4 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

In [ ]:
search_4.best_score_

In [ ]:
search_4.best_params_

In [ ]:
search_4.best_estimator_